# Genie Top200 Chart Crawling

In [1]:
import requests
import pandas as pd

### 1.인터넷에서 데이터 가져오기

In [5]:
# Genie Top 200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [12]:
# Chrome User-Agent
header = {'User-Agent':
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
}
req = requests.get(url, headers = header)
html = req.text
#html

In [16]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

### 찾으려고 하는 데이터 

In [22]:
trs = soup.find_all('tr')
trs

[<tr>
 <th class="hd-check" scope="col"><span class="hide">선택</span></th>
 <th class="hd-number" scope="col">순위</th>
 <th class="hd-album" scope="col"><span class="hide">앨범이미지</span></th>
 <th class="hd-link" scope="col"><span class="hide">곡정보 이동 링크</span></th>
 <th class="hd-info" scope="col">곡정보</th>
 <th class="hd-btns" scope="col">듣기</th>
 <th class="hd-btns" scope="col">추가</th>
 <th class="hd-btns" scope="col">담기</th>
 <th class="hd-btns" scope="col">다운</th>
 <th class="hd-btns" scope="col">뮤비</th>
 <th class="hd-more" scope="col">더보기</th>
 </tr>,
 <tr class="list" songid="95038278">
 <td class="check"><input class="select-check" title="리무진 (Feat. MINO) (Prod. by GRAY)" type="checkbox"/></td>
 <td class="number">1
                                         
                                     <span class="rank">
 <span class="rank"><span class="rank-none"><span class="hide">유지</span></span></span>
 </span>
 </td>
 <td><a class="cover" href="#" onclick="fnViewAlbumLayer('82398969');

In [21]:
len(trs[1:])

50

In [26]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)

50

In [33]:
# 개발자 도구에서 찾기
tbody = soup.select_one('#body-content > div.newest-list > div > table > tbody')
trs = tbody.select('tr')
len(trs)

50

In [34]:
# XPath 값은 아래와 같으나 BeautifulSoup은 지원하지 않음. 셀레니움에서 가능
//*[@id="body-content"]/div[6]/div/table/tbody                    

SyntaxError: invalid syntax (<ipython-input-34-70a7aaf09d5e>, line 1)

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추출
- ### rank, title, artist, album 추출

In [39]:
tr = trs[0]

In [57]:
# rank
tr.select_one('.number').get_text().split('\n')[0]

'1'

In [71]:
# title
tr.select_one('.title.ellipsis').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                리무진 (Feat. MINO) (Prod. by GRAY)'

In [76]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'리무진 (Feat. MINO) (Prod. by GRAY)'

In [80]:
# artist
artist = tr.select_one('.artist.ellipsis').get_text().strip()   #strip 해준 이유는 예방차원
artist

"BE'O (비오)"

In [85]:
# album
album = tr.select_one('.albumtitle.ellipsis').get_text()
album

'쇼미더머니 10 Episode 3'

### 4. 한 페이지에 있는 모든 데이터를 반복문으로 가져와 데이터 프레임 만들기

In [106]:
# 방법 1. 각 리스트에 담기
rank_list,title_list,artist_list,album_list = [],[],[],[]
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    

In [107]:
df = pd.DataFrame({
    '순위':rank_list,
    '제목':title_list,
    '가수':artist_list,
    '앨범':album_list
})
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [108]:
# 방법 2. 한 리스트에 다 담기
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text()
    lines.append([rank,title,artist,album])

In [109]:
dh = pd.DataFrame(lines)
dh.head()

,0,1,2,3
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [103]:
dh = pd.DataFrame(lines, columns=['순위','제목','가수','앨범'])
dh.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [110]:
# 방법 3. 
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text()
    d = {'순위':rank,'제목':title,'가수':artist,'album':album}
    lines.append(d)   

In [111]:
dk = pd.DataFrame(lines)
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


### 5. 모든 페이지의 데이터 가져오기

In [113]:
lines = []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211206&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')

    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text()
        lines.append([rank,title,artist,album])


In [114]:
di = pd.DataFrame(lines, columns=['순위','제목','가수','앨범'])
di

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
...,...,...,...,...
195,196,＋82 (Prod. by 다이나믹 듀오 & THAMA),팔로알토 (Paloalto) & 기리보이 & 다이나믹 듀오 & 저스디스 (JUSTH...,쇼미더머니 10 Final
196,197,DINOSAUR,AKMU (악뮤),SUMMER EPISODE
197,198,추적이는 여름 비가 되어,장범준,장범준 싱글
198,199,좋은 사람 있으면 소개시켜줘,조이 (JOY),슬기로운 의사생활 OST Part 2 (tvN 목요드라마)


In [115]:
di.to_csv('Genie Top 200(20211206).csv',index=False)